In [12]:
import arcpy
import os
import statistics

In [5]:
gdb = r"E:\OneDrive - Esri\GeoSpatial Authorities\Malbork_DL\Malbork_DL.gdb"
arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True
features_output = ["Buildings_USA_20","Buildings_USA_40","Buildings_USA_40","Buildings_USA_50"]
out_fc_name = "Buildings_USA"

In [14]:
arcpy.management.Merge(features_output, "Buildings_USA_Merged", "", "ADD_SOURCE_INFO")
arcpy.management.Dissolve("Buildings_USA_Merged", "Buildings_USA_Dissolved", "", "", "SINGLE_PART", "DISSOLVE_LINES")
arcpy.analysis.SpatialJoin("Buildings_USA_Merged", "Buildings_USA_Dissolved", "Buildings_USA_SpatialJoin", "JOIN_ONE_TO_MANY", "KEEP_ALL", "", "INTERSECT")

<Result 'E:\\OneDrive - Esri\\GeoSpatial Authorities\\Malbork_DL\\Malbork_DL.gdb\\Buildings_USA_SpatialJoin'>

In [15]:
# Create a dictionary to store the shape_area values for each JOIN_FID
shape_areas = {}

# Use a SearchCursor to iterate over the features
with arcpy.da.SearchCursor(f"{out_fc_name}_SpatialJoin", ["JOIN_FID", "shape_area"]) as cursor:
    for row in cursor:
        # Add the shape_area value to the list associated with the JOIN_FID
        shape_areas.setdefault(row[0], []).append(row[1])

# Create a dictionary to store the mean and standard deviation of shape_area values for each JOIN_FID
stats_areas = {join_fid: (statistics.mean(areas), statistics.stdev(areas)) for join_fid, areas in shape_areas.items() if len(areas) > 1}

merge_src_dict = {}
# Use a SearchCursor to iterate over the features again
with arcpy.da.SearchCursor(f"{out_fc_name}_SpatialJoin", ["OBJECTID", "JOIN_FID", "shape_area", "TARGET_FID_1", "MERGE_SRC"]) as cursor:
    for row in cursor:
        # If the absolute difference between the shape_area value and the mean is greater than 2 standard deviations, add the OBJECTID to the list
        join_fid = row[1]
        if join_fid in stats_areas:
            mean, stdev = stats_areas[join_fid]
            if abs(row[2] - mean) > 1.75 * stdev:
                        merge_src_dict.setdefault(row[4], []).append(row[3])

In [20]:
# Iterate over the items in the dictionary
for fc_path in merge_src_dict:
    # Use an UpdateCursor to iterate over the features in the feature class
    cursor = arcpy.da.UpdateCursor(fc_path, "OBJECTID")
    for row in cursor:
        # If the OBJECTID is in the list, delete the row
        if row[0] in merge_src_dict[fc_path]:
            print("hi")
    del cursor

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
